In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
import sklearn.linear_model as sk_lin_mod
import math
import seaborn as sns
import os
#from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
#os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence
from scipy import stats
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression

import PyQt5 #works
from PyQt5 import QtCore, QtGui, QtWidgets #works for pyqt5
%matplotlib qt

In [2]:
##### lecture du CSV
os.getcwd()
print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
##type(currDir)

donnee = pd.read_csv(fullPath)
stats=donnee.describe()
dimensions=donnee.shape
stats

/home/hadoop/Git_Ia/CegepSteFoy_IA/04_Eda/TP02


,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Pixel6_B,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B
count,1875.000000,1875.000000,1873.00000,1875.000000,1873.000000,1875.000000,1872.000000,1875.000000,1875.000000,1875.000000,...,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000,1875.000000
mean,2010.000000,45.144000,1.51543,-8.633867,-3.557768,10.147680,2.738568,610.334347,20.633067,26.523733,...,33.771435,157.936992,71.317269,32.709104,156.869221,70.605899,33.460053,157.787557,70.609437,32.820418
std,6.059559,25.781811,7.63530,8.695859,7.843854,4.770659,5.574896,126.965290,5.251394,10.234991,...,26.416038,85.971146,52.296358,26.692262,85.231625,52.970868,27.297388,85.760582,53.761268,27.040843
min,2000.000000,1.000000,-22.60000,-34.800000,-25.700000,0.900000,0.000000,313.200000,14.000000,11.567238,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,23.000000,-3.60000,-15.100000,-9.300000,6.700000,0.000000,536.400000,16.000000,17.280176,...,8.345000,111.895000,32.555000,4.865000,111.960000,31.290000,5.545000,109.940000,28.830373,4.326591
50%,2010.000000,45.000000,1.70000,-6.700000,-2.500000,9.600000,0.000000,620.300000,20.000000,26.462002,...,32.970000,183.190000,71.920000,31.110000,180.810000,69.960000,31.410000,183.020000,70.558732,31.111611
75%,2015.000000,67.000000,6.40000,-1.450000,2.400000,13.100000,2.600000,685.100000,25.000000,34.315361,...,55.820000,227.420000,102.465000,54.910000,224.955000,103.015000,56.400000,225.820000,104.848430,55.027608
max,2020.000000,90.000000,27.50000,10.000000,18.800000,31.000000,41.600000,811.700000,30.000000,46.784424,...,108.800000,255.000000,255.000000,125.400000,255.000000,255.000000,117.940000,255.000000,255.000000,132.121595


In [3]:
donnee.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          2
Temp min.(°C)                          0
Temp moy.(°C)                          2
Diff Temp (°C)                         0
Précip. tot. (mm)                      3
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Pixel1_R                               0
Pixel1_G                               0
Pixel1_B        

Heureusement, très peu de nulls.

Quels sont les enregistrements contenant des nulls?


In [4]:
donnee[(donnee['Temp max.(°C)'].isna()==True) | (donnee['Précip. tot. (mm)'].isna()==True)] 

,Date,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,...,Pixel7_R,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B,Classe Sirop
1519,2017-02-02,2017,2,NaN,-17.4,NaN,17.4,NaN,770.6,20,...,224.35,173.08,30.45,255.00,127.83,77.18,255.00,127.017170,10.139073,Ambré
1524,2017-02-07,2017,7,NaN,-25.1,NaN,25.1,NaN,770.6,20,...,255.00,131.77,3.27,225.25,123.51,61.07,194.07,70.795317,71.603721,Foncé
1632,2018-02-26,2018,26,3.2,-7.4,-2.1,10.6,NaN,811.7,16,...,213.40,111.76,3.39,212.26,98.49,35.60,193.85,127.018875,37.339503,Foncé


In [5]:
##### Comme il y a peu de null, on les remplit avec la valeur précédente (bfill)

donnee.fillna(method='bfill', inplace=True)
donnee.isnull().sum()

Date                                   0
Année                                  0
Jour Calendrier Saison                 0
Temp max.(°C)                          0
Temp min.(°C)                          0
Temp moy.(°C)                          0
Diff Temp (°C)                         0
Précip. tot. (mm)                      0
Précip. Tot. Hiver (mm)                0
Nombre épisodes gel/dégel              0
Production moyenne par entaille (L)    0
Débit sève (L/j)                       0
Sucre sève (%)                         0
Alimentation osmoseur (L/j)            0
Osmoseur (heures opération/j)          0
Pression osmoseur (bar)                0
Sucre sortie osmoseur (%)              0
Température Bouilloire (0C)            0
Temps bouilloire (h)                   0
Sucre du sirop obtenu (%)              0
Quantité de sirop obtenue (L)          0
Transmittance produit (%)              0
Pixel1_R                               0
Pixel1_G                               0
Pixel1_B        

In [6]:
donnee.columns

Index(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)',
       'Pixel1_R', 'Pixel1_G', 'Pixel1_B', 'Pixel2_R', 'Pixel2_G', 'Pixel2_B',
       'Pixel3_R', 'Pixel3_G', 'Pixel3_B', 'Pixel4_R', 'Pixel4_G', 'Pixel4_B',
       'Pixel5_R', 'Pixel5_G', 'Pixel5_B', 'Pixel6_R', 'Pixel6_G', 'Pixel6_B',
       'Pixel7_R', 'Pixel7_G', 'Pixel7_B', 'Pixel8_R', 'Pixel8_G', 'Pixel8_B',
       'Pixel9_R', 'Pixel9_G', 'Pixel9_B', 'Classe Sirop'],
      dtype='object')

Supprimer les colonnes pixel

In [7]:
### Pour la question 1, on va ignorer les colonnes Pixel

donnee.filter(like='Pixel').columns
dfSansPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1)
dfSansPixel.columns

Index(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)',
       'Classe Sirop'],
      dtype='object')

In [8]:
dfSansPixel.dtypes

Date                                    object
Année                                    int64
Jour Calendrier Saison                   int64
Temp max.(°C)                          float64
Temp min.(°C)                          float64
Temp moy.(°C)                          float64
Diff Temp (°C)                         float64
Précip. tot. (mm)                      float64
Précip. Tot. Hiver (mm)                float64
Nombre épisodes gel/dégel                int64
Production moyenne par entaille (L)    float64
Débit sève (L/j)                       float64
Sucre sève (%)                         float64
Alimentation osmoseur (L/j)            float64
Osmoseur (heures opération/j)          float64
Pression osmoseur (bar)                float64
Sucre sortie osmoseur (%)              float64
Température Bouilloire (0C)            float64
Temps bouilloire (h)                   float64
Sucre du sirop obtenu (%)                int64
Quantité de sirop obtenue (L)          float64
Transmittance

In [9]:
dfSansPixel.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
count,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.51,-8.63,-3.57,10.15,2.74,610.33,20.63,26.52,...,2.06,1038.45,2.34,34.06,6.84,88.52,2.02,57.02,74.69,37.08
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,0.94,1745.36,3.93,14.39,3.42,37.10,1.72,23.91,126.71,18.92
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,2.00,121.00,0.27,37.72,5.72,103.82,1.53,66.00,7.86,29.81
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,2.34,276.00,0.62,39.60,7.60,104.03,1.62,67.00,19.20,40.70
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,2.67,669.00,1.50,41.12,9.12,104.21,1.69,68.00,44.90,49.60
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,3.21,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52


In [10]:
## convertir Classe Sirop en numérique - https://www.youtube.com/watch?v=wH_ezgftiy0&t=136s
#dfSansPixel['CategClasseSirop'] = dfSansPixel['Classe Sirop'].astype('category').cat.codes
dfNumerique = dfSansPixel.select_dtypes(exclude='object').copy()
dfNumerique.describe().apply(lambda x: round(x, 2))

,Année,Jour Calendrier Saison,Temp max.(°C),Temp min.(°C),Temp moy.(°C),Diff Temp (°C),Précip. tot. (mm),Précip. Tot. Hiver (mm),Nombre épisodes gel/dégel,Production moyenne par entaille (L),...,Sucre sève (%),Alimentation osmoseur (L/j),Osmoseur (heures opération/j),Pression osmoseur (bar),Sucre sortie osmoseur (%),Température Bouilloire (0C),Temps bouilloire (h),Sucre du sirop obtenu (%),Quantité de sirop obtenue (L),Transmittance produit (%)
count,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,...,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00,1875.00
mean,2010.00,45.14,1.51,-8.63,-3.57,10.15,2.74,610.33,20.63,26.52,...,2.06,1038.45,2.34,34.06,6.84,88.52,2.02,57.02,74.69,37.08
std,6.06,25.78,7.64,8.70,7.84,4.77,5.57,126.97,5.25,10.23,...,0.94,1745.36,3.93,14.39,3.42,37.10,1.72,23.91,126.71,18.92
min,2000.00,1.00,-22.60,-34.80,-25.70,0.90,0.00,313.20,14.00,11.57,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2005.00,23.00,-3.60,-15.10,-9.30,6.70,0.00,536.40,16.00,17.28,...,2.00,121.00,0.27,37.72,5.72,103.82,1.53,66.00,7.86,29.81
50%,2010.00,45.00,1.70,-6.70,-2.50,9.60,0.00,620.30,20.00,26.46,...,2.34,276.00,0.62,39.60,7.60,104.03,1.62,67.00,19.20,40.70
75%,2015.00,67.00,6.40,-1.45,2.40,13.10,2.60,685.10,25.00,34.32,...,2.67,669.00,1.50,41.12,9.12,104.21,1.69,68.00,44.90,49.60
max,2020.00,90.00,27.50,10.00,18.80,31.00,41.60,811.70,30.00,46.78,...,3.21,10664.62,24.00,46.59,14.59,104.69,11.48,70.00,681.99,82.52


In [11]:
#### enelever les données où débit nul de sève
#dfNumerique = dfNumerique.loc[dfNumerique['Pression osmoseur (bar)'] !=0]
dfNumerique = dfNumerique.loc[dfNumerique['Débit sève (L/j)'] !=0]

In [12]:
### convertir Classe Sirop en var numérique
###
# Creating numeric columns
###
colsCategSirop = pd.get_dummies(dfSansPixel['Classe Sirop'], prefix='categSirop_', drop_first=True)

## concatenate side by side
dfNumerique = pd.concat([dfNumerique, colsCategSirop], axis=1)
#dfNumeriue.columns


In [13]:
dfNumerique.columns

Index(['Année', 'Jour Calendrier Saison', 'Temp max.(°C)', 'Temp min.(°C)',
       'Temp moy.(°C)', 'Diff Temp (°C)', 'Précip. tot. (mm)',
       'Précip. Tot. Hiver (mm)', 'Nombre épisodes gel/dégel',
       'Production moyenne par entaille (L)', 'Débit sève (L/j)',
       'Sucre sève (%)', 'Alimentation osmoseur (L/j)',
       'Osmoseur (heures opération/j)', 'Pression osmoseur (bar)',
       'Sucre sortie osmoseur (%)', 'Température Bouilloire (0C)',
       'Temps bouilloire (h)', 'Sucre du sirop obtenu (%)',
       'Quantité de sirop obtenue (L)', 'Transmittance produit (%)',
       'categSirop__Ambré', 'categSirop__Doré', 'categSirop__Foncé',
       'categSirop__Très Foncé'],
      dtype='object')

In [14]:
### Standardization
## https://datagy.io/pandas-normalize-column/
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(dfNumerique)
scaled = scaler.fit_transform(dfNumerique)
dfNumeriqueStd = pd.DataFrame(scaled, columns=dfNumerique.columns)

In [15]:
## book Practical Statiscs for Data Scientists, cahp 2
## https://github.com/gedeck/practical-statistics-for-data-scientists/blob/master/python/notebooks/Chapter%202%20-%20Data%20and%20sampling%20distributions.ipynb
def plot_prob_var_y(pDf):
    for y in list_cols_dependennt_vars:
        col_y = pDf[y][pDf[y]>0]
        print("y", y)
        #print("col[y]", dfNumerique[y][dfNumerique[y]>0])
        np.diff(np.log(col_y))
    
        fig, ax = plt.subplots(figsize=(4, 4))
        ax.set_title( y)
        sts.probplot(col_y, plot=ax)

In [16]:
def plot_box_plot_vars_y(pDf):
    for y in list_cols_dependennt_vars:
        col_y = pDf[y][pDf[y]>0]
        print("y", y)
    
        plt.figure()
        sns.boxplot(y=col_y).set_title(y)
        plt.show(block=False)

In [17]:
#plot_box_plot_vars_y(dfNumerique)

In [18]:
len(dfNumerique)

1875

---
Si on enlève seulement les outliers Débit de sève', on enlève plus que 1/3 du dataset.
Décision: on laisse tel que

---

### Multiple Linear Regression


https://www.geeksforgeeks.org/ml-multiple-linear-regression-using-python/

Assumption of Regression Model : 

- Linearity: The relationship between dependent and independent variables should be linear.
- Homoscedasticity: Constant variance of the errors should be maintained.
- Multivariate normality: Multiple Regression assumes that the residuals are normally distributed.
- Lack of Multicollinearity: It is assumed that there is little or no multicollinearity in

### Séparation des variables dépendantes (Y) et indépendantes (X)

In [19]:
### liste de variables (colonnes) dépendantes
list_cols_dependennt_vars = []
col_debit_seve = dfSansPixel['Débit sève (L/j)'].name
col_sucre_dans_seve = dfSansPixel['Sucre sève (%)'].name
col_pct_transmittance = dfSansPixel['Transmittance produit (%)'].name
col_productivite_seve_par_saison = dfSansPixel['Production moyenne par entaille (L)'].name
list_cols_dependennt_vars = [col_debit_seve, col_sucre_dans_seve, col_pct_transmittance, col_productivite_seve_par_saison]

In [20]:
## seulement les variables X
dfNumerique_X_cols = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

lstColTemp = ['Temp max.(°C)', 'Temp min.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:,~dfNumerique_X_cols.columns.isin(lstColTemp) ].copy()

## seulement les variables Y
dfNumerique_Y_cols = dfNumerique.loc[:, dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

len(dfNumerique.columns)

25

In [21]:
#### Après coup, on a appris que ces 2 variables causaient un prob de haute colinéarité.
#### et si on les enlève plus tôt?

lstColsExlude = ['Précip. Tot. Hiver (mm)', 'Temp moy.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin([col for col in lstColsExlude])]

#### Evaluate model

=============== Evaluating model ===========

https://www.youtube.com/watch?v=VCVhwjbI6h8

========

In [ ]:
### Fonction qui imprime le sommaire du modèle
def print_model_summary(pDfColsX, pDfColsY):
    ## Add constant
    pDfColsX = stm.add_constant(pDfColsX)
    lst_models = []
    for y in (pDfColsY.columns.values):
    
        col_y = dfNumerique_Y_cols[y]
    
        model = stm.OLS(col_y, pDfColsX.assign(const=1)).fit()
        print("=========")
        print("var depend: ", y)
    
        #lst_models.append(model)
        print(model.summary())
        #sns.distplot(model.resid, fit=sts.norm)        
        #p = sns.histplot(model.resid, kde=True, stat="density").set(title = y)
        

In [ ]:
print("---- Sommaire dfNumerique ----")
print_model_summary(dfNumerique_X_cols, dfNumerique_Y_cols)

In [ ]:
### https://github.com/simaria22/prediction_heart_failure/blob/master/feature_select.py
## Ajoute des colonnes au modèle pendant que la nouvelle colonne améliore
## le modèle OU
## jusqu'à qu'un certain niveau de précision soit atteint
def get_reduced_df_X(pDfx, pY, pAccuracy=0.7):
    print("------")
    print("dependent var y: ", pY.name)
    
    # new X df with fewer columns than pDfx
    reducedDf = []

    # column names to keep
    dfKeeper = None

    #no of features
    nof_list=np.arange(1,len(pDfx.columns))            
    high_score=0
    #Variable to store the optimum features
    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(pDfx, pY, test_size = 0.3, random_state = 0)
        model = LinearRegression()
        rfe = RFE(model,n_features_to_select= nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)
        score = model.score(X_test_rfe,y_test)
        score_list.append(score)
        
        
        print("n: ", n)
        print("score: ", score)
        print("high_score: ", high_score)

        # print summaries for the selection of attributes
        print("rfe.support_: ",rfe.support_)
        print("rfe.ranking_: " ,rfe.ranking_)
        
        #dfKeeper = pd.DataFrame(rfe.support_,index=pDfx.columns,columns=['Rank'])
        data= {'colName': pDfx.columns, 'keep': rfe.support_ }
        dfKeeper = pd.DataFrame(data)
        dfKeeper = dfKeeper[dfKeeper["keep"]==True]
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
        if (score > pAccuracy):
            break

    
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))
    print("df columns to keep: ")
    #print("shape")
    #print(dfNumerique_X_cols[dfKeeper["colName"]].shape)
    print(dfKeeper)
    
    reducedDf = dfNumerique_X_cols[dfKeeper["colName"]]
    
    #print("reducedDf: ", reducedDf.head())
    #print("unique: ", reducedDf[dfKeeper["colName"]].unique())
    return reducedDf

In [ ]:
### https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

dict_rfecv_x_y = {}

def get_features(pX, py, pDf=dfNumerique):
    min_features_to_select = 1  # Minimum number of features to consider
    clf = LogisticRegression()
    cv = KFold(10)
    clf_rf_4 = RandomForestRegressor()
    rfecv = RFECV(
        estimator=clf_rf_4,
        step=1,
        cv=cv,
        #scoring="accuracy",
        scoring="r2",
        min_features_to_select=min_features_to_select,
        n_jobs=-1,
    )
    rfecv.fit(pX, py)
    #print("indpendent var [pColName]: ", pColName)
    print(f"Optimal number of features: {rfecv.n_features_}")
    print('Best features :', pX.columns[rfecv.support_])
    #dict_rfecv_x_y.update({pColName: pX.columns[rfecv.support_]})
    return list(map(lambda col:pDf[col],  pX.columns[rfecv.support_]))

In [ ]:
### création d'un dictionnaire contenant la variable dépendantes et ses variables indépendantes
### correspondantes
### clé = colonne dépendante, valeur = dataframe des colonnes avec haute influence
dict_y_colsX = {}

for y in (dfNumerique_Y_cols.columns.values):    
    col_y = dfNumerique_Y_cols[y]
    reducedDf = get_reduced_df_X(dfNumerique_X_cols, col_y)
    #print("reduceDf.columns.values: ",  reducedDf.columns.values)
    #print("unique values: ", reducedDf[reducedDf.columns.values[0]].unique())
    #print("len: ", len(reducedDf[reducedDf.columns.values[0]]))
    #print("Nan: ", reducedDf[reducedDf.columns.values[0]].isnull().sum())
    ## call to analyze_model
    #analyze_model(dfNumerique_Y_cols, reducedDf)
    #analyze_model(col_y.to_frame(), reducedDf)
    dict_y_colsX.update({col_y.name: reducedDf})

In [ ]:
### création d'un dictionnaire contenant la variable dépendantes et ses variables indépendantes
### correspondantes
### clé = colonne dépendante, valeur = dataframe des colonnes avec haute influence
dict_y_colsX_refcv = {}

for y in (dfNumerique_Y_cols.columns.values):    
    print("[y] = ", y)
    col_y = dfNumerique_Y_cols[y]
    dict_y_colsX_refcv.update({col_y.name: get_features(dfNumerique_X_cols, col_y)})
    #reducedDf = get_reduced_df_X(dfNumerique_X_cols, col_y)
    #print("reduceDf.columns.values: ",  reducedDf.columns.values)
    #print("unique values: ", reducedDf[reducedDf.columns.values[0]].unique())
    #print("len: ", len(reducedDf[reducedDf.columns.values[0]]))
    #print("Nan: ", reducedDf[reducedDf.columns.values[0]].isnull().sum())
    ## call to analyze_model
    #analyze_model(dfNumerique_Y_cols, reducedDf)
    #analyze_model(col_y.to_frame(), reducedDf)
    

In [ ]:
#dfSansPixel.describe().apply(lambda x: round(x, 2))
#list(dict_y_colsX['Production moyenne par entaille (L)'])
#list(map(lambda col: dfNumerique[col], dict_y_colsX['Production moyenne par entaille (L)']))

### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX_refcv.keys()):
    print("[key]: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, pd.DataFrame(dfY))

In [ ]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_colsX.keys()):
    print("[key]: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, dfY.to_frame())

-----

À ce point, on a un modèle avec une bonne précision (R-squared et Adj R-squared) pour 
toutes les variables dépendantes.


-----

In [ ]:

#### RÉPONSES AUX QUESTIONS Q1


### Les tableaux ci-bas donnent, entre autres les informations concernant le adj. r-squared
###, t-value, p-value
### et coefficients des modèles.
### !!!NOTE!!! générer les graphiques et le sauvegarder à part!!!

In [ ]:
### pairplots
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
i=0
for key in (dict_y_colsX.keys()):
    i+=1
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    #sns.pairplot(pd.concat([dfX, dfY], axis=0))
    #plt.figure(i)
    #print(dfX.columns.values)
    #print(dfNumerique[key].head())
    #plt.figure()
    #plt.plot(pd.concat([dfX, dfY], axis=0))
    sns.pairplot(data = dfNumerique, x_vars = dfX.columns.values , y_vars = [key], kind='reg', diag_kind=None)
    
    #sns.pairplot(data = reducedDf, x_vars = ['Nombre épisodes gel/dégel'], y_vars = ['Production moyenne par entaille (L)'], kind='reg', diag_kind='kde')
    #plt.show(block=False)

    

In [ ]:
### plot regressions
#https://seaborn.pydata.org/tutorial/regression.html
i=0
for key in (dict_y_colsX.keys()):
    
    print("key: ", key)
    dfX = dict_y_colsX.get(key)
    dfY = dfNumerique_Y_cols[key]    
    for col_x in dfX.columns.values:
        i+=1
        plt.figure(i)
        print("col_x: ", col_x)
        sns.regplot(data = dfNumerique, x = col_x, y = key)    
        plt.show(block=False)

key:  Production moyenne par entaille (L)
col_x:  Nombre épisodes gel/dégel


TypeError: object of type 'NoneType' has no len()

In [ ]:
#### END Evaluate model

In [ ]:
from featurewiz import featurewiz

In [ ]:
### fonction pour obtenir un subset d'un dataset contenant seulement les variables indépendantes (x)
def get_x_vars(pDf, pVarY):
    return pDf.drop([pVarY], axis=1)

In [ ]:
### retourne un dictionnaire dont key=var dépendante (y) et valeur = dataframe de var x (indépenantes)
def get_dict_y_x(pDf, pListY):
    dict_y_x = {}
    for key in pListY:        
        dict_y_x.update({key: get_x_vars(pDf, key)})
    return dict_y_x

In [ ]:
### dictionnaire contenant les variables dépendantes (y) comme clé et les indépendantes comme df
dict_y_x = get_dict_y_x(dfNumerique, list_cols_dependennt_vars)

dict_y_x_featurewiz = {}
for key in dict_y_x:
    print("------------------")
    print("[key]: ", key)
    
    features = featurewiz(dfNumerique, target=key, corr_limit=0.70, 
                                verbose=0) ## verbose = 2 -> generates graph    
    dict_y_x_featurewiz.update({key: features[1]}) ## features[1] = dataframe

In [ ]:
### analyse le modèle de nouveau avec un dataframe réduit
#analyze_model(dfNumerique_Y_cols, dfNumerique_X_cols)
for key in (dict_y_x_featurewiz.keys()):
    print("[key]: ", key)
    dfX = dict_y_x_featurewiz.get(key)
    dfY = dfNumerique_Y_cols[key]    
    print_model_summary(dfX, dfY.to_frame())